# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 14 2022 11:09AM,253090,15,30012876,"Alliance Pharma, Inc.",Released
1,Dec 14 2022 11:09AM,253090,15,30012877,"Alliance Pharma, Inc.",Released
2,Dec 14 2022 11:09AM,253090,15,30012878,"Alliance Pharma, Inc.",Released
3,Dec 14 2022 11:09AM,253090,15,30012879,"Alliance Pharma, Inc.",Released
4,Dec 14 2022 11:09AM,253090,15,30012880,"Alliance Pharma, Inc.",Released
5,Dec 14 2022 11:09AM,253090,15,30012881,"Alliance Pharma, Inc.",Released
6,Dec 14 2022 11:09AM,253090,15,30012882,"Alliance Pharma, Inc.",Released
7,Dec 14 2022 11:09AM,253090,15,30012883,"Alliance Pharma, Inc.",Released
8,Dec 14 2022 11:09AM,253090,15,30012884,"Alliance Pharma, Inc.",Released
9,Dec 14 2022 11:09AM,253090,15,30012885,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,253088,Released,1
25,253089,Released,1
26,253090,Released,43
27,253091,Released,2
28,253092,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253088,NaN,NaN,1.0
253089,NaN,NaN,1.0
253090,NaN,NaN,43.0
253091,NaN,NaN,2.0
253092,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252982,0.0,0.0,1.0
253002,0.0,0.0,13.0
253003,0.0,0.0,10.0
253035,0.0,0.0,1.0
253049,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252982,0,0,1
253002,0,0,13
253003,0,0,10
253035,0,0,1
253049,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252982,0,0,1
1,253002,0,0,13
2,253003,0,0,10
3,253035,0,0,1
4,253049,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252982,,,1
1,253002,,,13
2,253003,,,10
3,253035,,,1
4,253049,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 14 2022 11:09AM,253090,15,"Alliance Pharma, Inc."
43,Dec 14 2022 11:08AM,253092,10,Emerginnova
44,Dec 14 2022 11:07AM,253091,15,"Virtus Pharmaceuticals, LLC"
46,Dec 14 2022 10:55AM,253089,20,"Adare Pharmaceuticals, Inc."
47,Dec 14 2022 10:51AM,253088,19,"GUSA Granules USA, Inc."
48,Dec 14 2022 10:48AM,253086,19,"GUSA Granules USA, Inc."
49,Dec 14 2022 10:44AM,253064,10,ISDIN Corporation
72,Dec 14 2022 10:07AM,253083,10,Bio-PRF
85,Dec 14 2022 10:00AM,253082,10,Bio-PRF
86,Dec 14 2022 9:51AM,253081,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 14 2022 11:09AM,253090,15,"Alliance Pharma, Inc.",,,43
1,Dec 14 2022 11:08AM,253092,10,Emerginnova,,,1
2,Dec 14 2022 11:07AM,253091,15,"Virtus Pharmaceuticals, LLC",,,2
3,Dec 14 2022 10:55AM,253089,20,"Adare Pharmaceuticals, Inc.",,,1
4,Dec 14 2022 10:51AM,253088,19,"GUSA Granules USA, Inc.",,,1
5,Dec 14 2022 10:48AM,253086,19,"GUSA Granules USA, Inc.",,,1
6,Dec 14 2022 10:44AM,253064,10,ISDIN Corporation,,,23
7,Dec 14 2022 10:07AM,253083,10,Bio-PRF,,,13
8,Dec 14 2022 10:00AM,253082,10,Bio-PRF,,,1
9,Dec 14 2022 9:51AM,253081,10,Bio-PRF,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 11:09AM,253090,15,"Alliance Pharma, Inc.",43,,
1,Dec 14 2022 11:08AM,253092,10,Emerginnova,1,,
2,Dec 14 2022 11:07AM,253091,15,"Virtus Pharmaceuticals, LLC",2,,
3,Dec 14 2022 10:55AM,253089,20,"Adare Pharmaceuticals, Inc.",1,,
4,Dec 14 2022 10:51AM,253088,19,"GUSA Granules USA, Inc.",1,,
5,Dec 14 2022 10:48AM,253086,19,"GUSA Granules USA, Inc.",1,,
6,Dec 14 2022 10:44AM,253064,10,ISDIN Corporation,23,,
7,Dec 14 2022 10:07AM,253083,10,Bio-PRF,13,,
8,Dec 14 2022 10:00AM,253082,10,Bio-PRF,1,,
9,Dec 14 2022 9:51AM,253081,10,Bio-PRF,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 11:09AM,253090,15,"Alliance Pharma, Inc.",43,,
1,Dec 14 2022 11:08AM,253092,10,Emerginnova,1,,
2,Dec 14 2022 11:07AM,253091,15,"Virtus Pharmaceuticals, LLC",2,,
3,Dec 14 2022 10:55AM,253089,20,"Adare Pharmaceuticals, Inc.",1,,
4,Dec 14 2022 10:51AM,253088,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 11:09AM,253090,15,"Alliance Pharma, Inc.",43,NaN,NaN
1,Dec 14 2022 11:08AM,253092,10,Emerginnova,1,NaN,NaN
2,Dec 14 2022 11:07AM,253091,15,"Virtus Pharmaceuticals, LLC",2,NaN,NaN
3,Dec 14 2022 10:55AM,253089,20,"Adare Pharmaceuticals, Inc.",1,NaN,NaN
4,Dec 14 2022 10:51AM,253088,19,"GUSA Granules USA, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 14 2022 11:09AM,253090,15,"Alliance Pharma, Inc.",43,0.0,0.0
1,Dec 14 2022 11:08AM,253092,10,Emerginnova,1,0.0,0.0
2,Dec 14 2022 11:07AM,253091,15,"Virtus Pharmaceuticals, LLC",2,0.0,0.0
3,Dec 14 2022 10:55AM,253089,20,"Adare Pharmaceuticals, Inc.",1,0.0,0.0
4,Dec 14 2022 10:51AM,253088,19,"GUSA Granules USA, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3289730,110,9.0,0.0
15,1265334,66,20.0,0.0
16,253035,1,0.0,0.0
19,1012311,22,11.0,3.0
20,253089,1,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3289730,110,9.0,0.0
1,15,1265334,66,20.0,0.0
2,16,253035,1,0.0,0.0
3,19,1012311,22,11.0,3.0
4,20,253089,1,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,110,9.0,0.0
1,15,66,20.0,0.0
2,16,1,0.0,0.0
3,19,22,11.0,3.0
4,20,1,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,110.0
1,15,Released,66.0
2,16,Released,1.0
3,19,Released,22.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20
Status,,,,,
Completed,0.0,0.0,0.0,3.0,0.0
Executing,9.0,20.0,0.0,11.0,0.0
Released,110.0,66.0,1.0,22.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20
0,Completed,0.0,0.0,0.0,3.0,0.0
1,Executing,9.0,20.0,0.0,11.0,0.0
2,Released,110.0,66.0,1.0,22.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20
0,Completed,0.0,0.0,0.0,3.0,0.0
1,Executing,9.0,20.0,0.0,11.0,0.0
2,Released,110.0,66.0,1.0,22.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()